In [2]:
from owslib.wfs import WebFeatureService

In [3]:
wfs11 = WebFeatureService(url='https://www.imis.bfs.de/ogc/opendata/ows', version='1.1.0')
wfs11.identification.title

'BfS Download'

In [4]:
[operation.name for operation in wfs11.operations]

['GetCapabilities',
 'DescribeFeatureType',
 'GetFeature',
 'GetGmlObject',
 'LockFeature',
 'GetFeatureWithLock',
 'Transaction']

In [5]:
list(wfs11.contents)

['opendata:abi_30min',
 'opendata:abi_30min_barchart',
 'opendata:abi_30min_barchart_store',
 'opendata:abi_30min_timeseries',
 'opendata:bfg_gesamt_gamma',
 'opendata:bfg_sondenstandorte',
 'opendata:bfs_locations',
 'opendata:bsh_gesamt_gamma_kuenstlich',
 'opendata:bsh_gesamt_gamma_kuenstlich_timeseries',
 'opendata:bsh_sondenstandorte',
 'opendata:eurdep_latestValue',
 'opendata:eurdep_maxValue',
 'opendata:bfg_gesamt_gamma_timeseries',
 'opendata:new_dwd_alphabeta_kuenstl_24h',
 'opendata:new_dwd_alphabeta_kuenstl_24h_timeseries',
 'opendata:new_dwd_gamma_aerosole_24h_nuclide',
 'opendata:new_dwd_iod_24h',
 'opendata:new_dwd_iod_24h_timeseries',
 'opendata:new_dwd_niederschlag_24h',
 'opendata:new_dwd_niederschlag_24h_timeseries',
 'opendata:new_dwd_niederschlag_ges_beta_24h_verz',
 'opendata:new_dwd_niederschlag_ges_beta_24h_verz_timeseries',
 'opendata:new_gamma_aerosole_24h',
 'opendata:new_gamma_aerosole_24h_timeseries',
 'opendata:new_nuklide_bodenoberflaeche',
 'opendata:nuk

In [124]:
wfs11.get_schema('opendata:odl_brutto_1h')

{'properties': {'id': 'string',
  'locality_name': 'string',
  'start': 'dateTime',
  'end_measure': 'dateTime',
  'unit': 'string',
  'value_e': 'string',
  'value': 'decimal',
  'nuclide': 'string',
  'duration': 'string',
  'dom': 'string',
  'source': 'string',
  'network_id': 'string'},
 'required': ['id', 'locality_name', 'end_measure', 'network_id'],
 'geometry': 'Point',
 'geometry_column': 'geom'}

In [6]:

import xml.dom.minidom # WFS returns GML - this library is used to parse it
response = wfs11.getfeature(typename='opendata:nuklide_fleisch', bbox=(0.0004322,0.0000835,0.0004397,0.0001045), srsname='EPSG:4326')
xml = xml.dom.minidom.parse(response) # or xml.dom.minidom.parseString(xml_string)
print(xml.toprettyxml(indent="    "))

<?xml version="1.0" ?>
<wfs:FeatureCollection xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:wfs="http://www.opengis.net/wfs" xmlns:opendata="www.imis.bfs.de/opendata" xmlns:gml="http://www.opengis.net/gml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" numberOfFeatures="27" timeStamp="2022-07-17T21:16:41.377Z" xsi:schemaLocation="www.imis.bfs.de/opendata https://www.imis.bfs.de/ogc/opendata/wfs?service=WFS&amp;version=1.1.0&amp;request=DescribeFeatureType&amp;typeName=opendata%3Anuklide_fleisch http://www.opengis.net/wfs https://www.imis.bfs.de/ogc/schemas/wfs/1.1.0/wfs.xsd">
    <gml:featureMembers>
        <opendata:nuklide_fleisch gml:id="nuklide_fleisch.fid--3ee2bc79_1820e04a50d_45fd">
            <opendata:id>ZDB000000158284Y</opendata:id>
            <opendata:sample_begin>2021-07-19T08:00:00Z</opendata:sample_begin>
            <opendata:id_long>2021-07-19 08:00:00 (Probe: ZDB000000158284Y)</opendata:id_long>
            <opendata:sample_type>N54</opendata:sample_type

In [7]:
from owslib.fes import *
from owslib.etree import etree

filter = PropertyIsLike(nuclide='nuclide', literal='Ingolstadt', wildCard='*')
filterxml = etree.tostring(filter.toXML()).decode("utf-8")
response = wfs11.getfeature(typename='bvv:gmd_ex', filter=filterxml)

TypeError: PropertyIsLike.__init__() got an unexpected keyword argument 'nuclide'

In [8]:
from owslib.fes import *
from owslib.etree import etree


#filter = PropertyIsLike(propertyname='locality_name', literal='Lögow', wildCard='*')
#filter = PropertyIsLike(propertyname='nuclide', literal='Gamma-ODL-Brutto', wildCard='*')
filter = PropertyIsEqualTo(propertyname='value', literal='0.074', matchcase=True)
#filter = PropertyIsGreaterThanOrEqualTo(propertyname='end_measure', literal='2022-07-13 15:43:12.948085', matchcase=True)
filterxml = etree.tostring(filter.toXML()).decode("utf-8")


response = wfs11.getfeature(
    typename= 'opendata:odl_brutto_1h', 
    filter= filterxml,

)

out = open('/tmp/data.gml', 'wb')
out.write(bytes(response.read(), 'UTF-8'))
out.close()

KeyboardInterrupt: 